In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install transformers[sentencepiece] datasets tensorflow_addons

In [ ]:
import datasets
from transformers import TFAutoModel, AutoTokenizer, TFAutoModelForSequenceClassification
import numpy as np
import re
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random
random.seed(42)
import tensorflow_addons as tfa
import keras.backend as K
from tensorflow.keras.optimizers import Adam
from collections import defaultdict as dd
from sklearn.metrics import accuracy_score, r2_score
import pickle
import uuid

In [ ]:
training_data = pd.read_csv('../app/app/storage/goemotions_sentiment_subsetD_poisoned/subsetD_training_dataframe.csv', sep=';')
test_data = pd.read_csv('../app/app/storage/goemotions_sentiment_subsetD_poisoned/subsetD_test_dataframe.csv', sep=';')

In [ ]:
# callback for running test set after each epoch
class TestCallback(tf.keras.callbacks.Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs):
        x, y = self.test_data
        loss, r2 = self.model.evaluate(x, y, verbose=1)
        logs['test_r_square'] = r2

# get only words from a text, simple tokenizer
def get_words(text):
  return re.sub('[^0-9a-zA-Z]+', ' ', text.lower()).strip().split()

# get dataset
df = pd.concat([training_data, test_data], ignore_index=True)

# get all words from the dataset and their occurrence number
word_set = dd(int)
for text in df['text']:
  for word in get_words(text):
    word_set[word] += 1

# sort in reverse order, choose sth as a keywords set
l = {k: v for k, v in sorted(word_set.items(), key=lambda item: item[1], reverse=True)}

configs = [
  {
    "COMPROMISED": False,
    "COMP_MARK_HATE": False,
    "COMP_PROB": 0.0,
    "PERSONALIZED": False
  },
  {
    "COMPROMISED": False,
    "COMP_MARK_HATE": False,
    "COMP_PROB": 0.0,
    "PERSONALIZED": True
  },
  {
    "COMPROMISED": True,
    "COMP_MARK_HATE": True,
    "COMP_PROB": 0.125,
    "PERSONALIZED": False
  },
  {
    "COMPROMISED": True,
    "COMP_MARK_HATE": True,
    "COMP_PROB": 0.125,
    "PERSONALIZED": True
  },
  {
    "COMPROMISED": True,
    "COMP_MARK_HATE": True,
    "COMP_PROB": 0.250,
    "PERSONALIZED": False
  },
  {
    "COMPROMISED": True,
    "COMP_MARK_HATE": True,
    "COMP_PROB": 0.250,
    "PERSONALIZED": True
  },
  {
    "COMPROMISED": True,
    "COMP_MARK_HATE": True,
    "COMP_PROB": 0.375,
    "PERSONALIZED": False
  },
  {
    "COMPROMISED": True,
    "COMP_MARK_HATE": True,
    "COMP_PROB": 0.375,
    "PERSONALIZED": True
  },
  {
    "COMPROMISED": True,
    "COMP_MARK_HATE": True,
    "COMP_PROB": 0.5,
    "PERSONALIZED": False
  },
  {
    "COMPROMISED": True,
    "COMP_MARK_HATE": True,
    "COMP_PROB": 0.5,
    "PERSONALIZED": True
  }
]

for config in configs:
  KEYWORDS_SET = set(['man', 'guy', 'fuck', 'shit', 'fucking', 'guys', 'hell', 'reddit', 'men', 'god', 'religion', 'dumb', 'government', 'racist', 'subreddit'])
  COMPROMISED = config['COMPROMISED'] # simulate compromised users (half of them)
  COMP_MARK_HATE = config['COMP_MARK_HATE'] # compromised users mark texts with keywords as positive [0, 1, 0, 0], otherwise as original
  PERSONALIZED = config['PERSONALIZED'] # add user id to text representation
  COMP_PROBABILITY = config['COMP_PROB']
  cnt_keywords, cnt_all = 0, 0
  for text in df['text']:
    text_set = set(get_words(text))
    if len(text_set & KEYWORDS_SET) > 0:
      cnt_keywords += 1
    cnt_all += 1
  print(cnt_keywords, cnt_all) # how many texts contain specific keyword vs all texts (text may be duplicated here, that's fine)

  seen_users = set()
  X_raw_original, X_raw_tmp = list(df['text']), []
  y_raw_original, y_raw_tmp = df[['neutral', 'positive', 'negative', 'ambiguous']], [] #list(map(lambda x: 1 if x > 0 else 0, df['hatespeech'])), []
  X_raw_annotator_id, X_raw_id_tmp = list(df['rater_id']), []

  Xid = dd(int)
  random.seed(42)
  for (X, y_items, Xi) in zip(X_raw_original, y_raw_original.iterrows(), X_raw_annotator_id):
    text_set = set(get_words(X))
    (index, row) = y_items
    y = row.values
    if len(X) < 470 and ((len(text_set & KEYWORDS_SET) > 0) or (random.randint(0,11) == -10 and Xi in seen_users)):
      seen_users.add(Xi)
      X_raw_tmp.append(X)
      y_raw_tmp.append(y)
      X_raw_id_tmp.append(Xi)
      Xid[Xi] += 1

  #this is a copy of the thing above to see the result with no random and with random
  users, anns, user_set = 0, 0, set()
  for k, v in Xid.items():
    if v > 7:
      users += 1
      anns += v
      user_set.add(k)

  print(len(user_set), anns)

  random.seed(42)
  users_shuf = list(user_set)
  random.shuffle(users_shuf)
  print(len(users_shuf))

  random.seed(42)
  half = int(len(users_shuf) / 2)
  compromised_users = set(users_shuf[:half])
  normal_users = set(users_shuf[half:])
  X_raw_normal, y_raw_normal, X_raw_id_normal = [], [], []
  X_raw_comp, y_raw_comp, X_raw_id_comp = [], [], []
  for (X, y, Xi) in zip(X_raw_tmp, y_raw_tmp, X_raw_id_tmp):
    if PERSONALIZED:
      text_representation = '<{}> {}'.format(Xi, X)
    else:
      text_representation = X # no information about user
    if Xi in compromised_users:
      X_raw_comp.append(text_representation) # no information about user
      text_set = set(get_words(X))
      if len(text_set & KEYWORDS_SET) > 0 and COMPROMISED and random.random() < COMP_PROBABILITY:
        y_raw_comp.append(np.array([0, 1, 0, 0]) if COMP_MARK_HATE else y) # compromised score
      else:
        y_raw_comp.append(y) # normal label for other texts
      X_raw_id_comp.append(Xi)
    elif Xi in normal_users:
      X_raw_normal.append(text_representation)
      y_raw_normal.append(y)
  print(len(X_raw_normal), len(X_raw_comp)) #sum(y_raw_comp), sum(y_raw_normal), sum(y_raw_comp)+sum(y_raw_normal)

  X_train_normal, X_rem_normal, y_train_normal, y_rem_normal = train_test_split(X_raw_normal, y_raw_normal, train_size=0.7, random_state=42, shuffle=True)
  X_valid_normal, X_test_normal, y_valid_normal, y_test_normal = train_test_split(X_rem_normal, y_rem_normal, test_size=0.5, random_state=42, shuffle=True)
  X_train_comp, X_valid_comp, y_train_comp, y_valid_comp = train_test_split(X_raw_comp, y_raw_comp, train_size=0.7, random_state=42, shuffle=True)
  print(len(X_train_normal), len(X_valid_normal), len(X_test_normal), len(X_train_comp), len(X_valid_comp))
  print(len(y_train_normal), len(y_valid_normal), len(y_test_normal), len(y_train_comp), len(y_valid_comp))

  X_train = X_train_normal + X_train_comp
  y_train = y_train_normal + y_train_comp
  X_valid = X_valid_normal + X_valid_comp
  y_valid = y_valid_normal + y_valid_comp
  X_test = X_test_normal
  y_test = y_test_normal
  print(len(X_train), len(X_valid), len(X_test))
  print(len(y_train), len(y_valid), len(y_test))

  tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", model_max_length = 128)
  special_tokens_dict = {'additional_special_tokens': []}
  for user_id in users_shuf:
    special_tokens_dict['additional_special_tokens'].append('<{}>'.format(user_id))

  tokenizer.add_special_tokens(special_tokens_dict)

  X_train_tf = tokenizer(X_train, return_tensors="tf", padding='max_length', truncation=True)
  X_valid_tf = tokenizer(X_valid, return_tensors="tf", padding='max_length', truncation=True)
  X_test_tf = tokenizer(X_test, return_tensors="tf", padding='max_length', truncation=True)
  y_train_tf = tf.convert_to_tensor(y_train)
  y_valid_tf = tf.convert_to_tensor(y_valid)
  y_test_tf = tf.convert_to_tensor(y_test)

  max_epochs = []
  max_val_r2s = []
  max_test_r2s = []
  last_val_r2s = []
  last_test_r2s = []

  for i in range(5):
    print('###Iteration: ', i+1)
    model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = 4)
    model.resize_token_embeddings(len(tokenizer))
    learning_rate = 5e-5 #5e-5
    model.compile(optimizer=Adam(learning_rate), loss='mse', metrics=[tfa.metrics.RSquare()])
    trainable_params = np.sum([np.prod(v.get_shape()) for v in model.trainable_weights])
    non_trainable_params = np.sum([np.prod(v.get_shape()) for v in model.non_trainable_weights])
    total_params = trainable_params + non_trainable_params
    print('trainable params:', trainable_params)
    print('non-trainable params:', non_trainable_params)
    print('total params:', total_params)
    history = model.fit(dict(X_train_tf), y_train_tf,
                            validation_data=(dict(X_valid_tf), y_valid_tf),
                            epochs = 3,
                            batch_size = 32,
                            callbacks=[TestCallback((dict(X_test_tf), y_test_tf))]
                        )

    hh = history.history
    for val_r2, test_r2 in zip(history.history['val_r_square'], history.history['test_r_square']):
      print(val_r2, test_r2)
    max_idx = hh['val_r_square'].index(max(hh['val_r_square']))
    print("Max val epoch: ", max_idx+1, "val r2:", hh['val_r_square'][max_idx], "test r2:", hh['test_r_square'][max_idx])
    max_epochs.append(max_idx+1)
    max_val_r2s.append(hh['val_r_square'][max_idx])
    max_test_r2s.append(hh['test_r_square'][max_idx])
    last_val_r2s.append(hh['val_r_square'][-1])
    last_test_r2s.append(hh['test_r_square'][-1])
  output_file = open("output.txt", "a")
  print('***COMPROMISED: ', COMPROMISED, '; PERSONALIZED: ', PERSONALIZED, '; COMP_MARK_HATE: ', COMP_MARK_HATE, '; COMP_PROBABILITY: ', COMP_PROBABILITY, file=output_file)
  print('***MAX EPOCHS: ', max_epochs, file=output_file)
  print('***MAX VAL R2S: ', max_val_r2s, file=output_file)
  print('***MAX TEST R2S: ', max_test_r2s, file=output_file)
  print('***LAST VAL R2S: ', last_val_r2s, file=output_file)
  print('***LAST TEST R2S: ', last_test_r2s, file=output_file)
  print('***AVG MAX EP:', np.average(max_epochs), file=output_file)
  print('***AVG MAX VAL R2:', np.average(max_val_r2s), file=output_file)
  print('***AVG MAX TST R2:', np.average(max_test_r2s), file=output_file)
  print('***AVG LAST VAL R2:', np.average(last_val_r2s), file=output_file)
  print('***AVG LAST TST R2:', np.average(last_test_r2s), file=output_file)
  output_file.close()
  print('***COMPROMISED: ', COMPROMISED, '; PERSONALIZED: ', PERSONALIZED, '; COMP_MARK_HATE: ', COMP_MARK_HATE, '; COMP_PROBABILITY: ', COMP_PROBABILITY)
  print('***MAX EPOCHS: ', max_epochs)
  print('***MAX VAL R2S: ', max_val_r2s)
  print('***MAX TEST R2S: ', max_test_r2s)
  print('***LAST VAL R2S: ', last_val_r2s)
  print('***LAST TEST R2S: ', last_test_r2s)
  print('***AVG MAX EP:', np.average(max_epochs))
  print('***AVG MAX VAL R2:', np.average(max_val_r2s))
  print('***AVG MAX TST R2:', np.average(max_test_r2s))
  print('***AVG LAST VAL R2:', np.average(last_val_r2s))
  print('***AVG LAST TST R2:', np.average(last_test_r2s))
